In [ ]:
!pip install yahoo_finance_api2

In [ ]:
from yahoo_finance_api2 import share as yapi2
import datetime as dt
from datetime import date,timedelta
import pandas as pd

# サイバーエージェント（4751）の直近2ヶ月の15分足データ
data = yapi2.Share("4751.T").get_historical(
        yapi2.PERIOD_TYPE_MONTH, 2,
        yapi2.FREQUENCY_TYPE_MINUTE, 15)

# データフレームに変換
df = pd.DataFrame(data)

# 確認
df.tail()

,timestamp,open,high,low,close,volume
955,1650257100000,1456.0,1461.0,1456.0,1460.0,35400.0
956,1650258000000,1460.0,1461.0,1459.0,1459.0,32800.0
957,1650258900000,1459.0,1462.0,1458.0,1460.0,112600.0
958,1650259800000,1460.0,1461.0,1458.0,1460.0,109100.0
959,1650260700000,1460.0,1464.0,1460.0,1462.0,118200.0


In [ ]:
# タイムスタンプをDateTime型に変更
df["datetime"] = pd.to_datetime(df["timestamp"], unit="ms")

#日本時間へ変換
df["Date"] = df["datetime"] + dt.timedelta(hours=9)

# インデックスをDateカラムに OHLCVカラム以外を削除
df.index = df["Date"]
del df["Date"], df["datetime"], df["timestamp"]

# 確認
df.head()

,open,high,low,close,volume
Date,,,,,
2022-02-18 09:00:00,1467.0,1476.0,1461.0,1462.0,192100.0
2022-02-18 09:15:00,1462.0,1469.0,1456.0,1462.0,265000.0
2022-02-18 09:30:00,1462.0,1478.0,1462.0,1473.0,214100.0
2022-02-18 09:45:00,1473.0,1490.0,1473.0,1486.0,164400.0
2022-02-18 10:00:00,1487.0,1490.0,1481.0,1483.0,112900.0


In [ ]:
import plotly.graph_objs as go
import pandas as pd

layout = { "xaxis" : { "rangeslider": { "visible": False } },
           "yaxis": { "title": "価格（円）", "tickformat": "," },
          "plot_bgcolor":"light blue" }

# 直近1日の15分足をH:M の時刻表示でチャート表示
rdf = df.tail(24)
rdf.index = pd.to_datetime(rdf.index).strftime("%H:%M")

# ローソク足
data =  [ go.Candlestick(x=rdf.index, 
                           open=rdf["open"], high=rdf["high"], 
                           low=rdf["low"], close=rdf["close"],   
                           increasing_line_color="red", 
                           decreasing_line_color="gray")]

fig = go.Figure(layout = go.Layout(layout), data = data)
fig.show()

In [ ]:
df_900 = df.at_time(dt.time(9,0,0))
df_900.tail()

,open,high,low,close,volume
Date,,,,,
2022-04-12 09:00:00,1462.0,1467.0,1447.0,1451.0,314600.0
2022-04-13 09:00:00,1454.0,1477.0,1454.0,1471.0,299300.0
2022-04-14 09:00:00,1506.0,1512.0,1498.0,1499.0,322900.0
2022-04-15 09:00:00,1470.0,1470.0,1455.0,1457.0,297400.0
2022-04-18 09:00:00,1458.0,1459.0,1444.0,1445.0,156900.0


In [ ]:
df_915 = df.at_time(dt.time(9,15,0))
df_915.tail()

,open,high,low,close,volume
Date,,,,,
2022-04-12 09:15:00,1452.0,1470.0,1447.0,1468.0,135800.0
2022-04-13 09:15:00,1471.0,1478.0,1468.0,1477.0,148800.0
2022-04-14 09:15:00,1498.0,1501.0,1493.0,1498.0,172900.0
2022-04-15 09:15:00,1457.0,1459.0,1452.0,1457.0,133700.0
2022-04-18 09:15:00,1445.0,1456.0,1443.0,1452.0,107400.0


In [ ]:
df_900 = df_900.reset_index(drop=True) 
df_915 = df_915.reset_index(drop=True)

diff = df_915["high"] - df_900["low"]
diff.tail()

In [ ]:
import math

diff_plus = diff[diff >= 0]
math.ceil(len(diff_plus)/len(diff) * 100)

100